# Airports per country (please stop sending shapefiles)
Here you will learn...
- how to read shapefiles and csv files using python geopandas and then create PostGIS from this data.
- how to read shapefiles and csv files using GDAL ogr2ogr to then create PostGIS from this data.
- how to do a spatial query in PostGIS that counts points within polygons.
- how to turn this spatial query into a view - a vitual table - that automatically updates as the underlying data does.
- how to retrieve the data in QGIS to create a styled map.

## Scenario

![airport count and gdp per country](./story_images/pop_per_airport.PNG)

You are tasked by your boss to create a dataset which combines global airport and country data to display how many airports each country has. The underlying data will be updated on a weekly basis and it has to be easy to reflect these changes also on your dataset.

To that end she sends you a NaturalEarth country shapefile and a csv with the airports. It is quite common at your company to share shapefiles with your colleagues. This has led to problems in the past whereby data inconsistency was a result of coworkers not always using the most updated version of the data and general frustration arose from keeping track of email attachments and download links.

### The plan
You plan to make use of a spatial database which can be accessed by your coworkers to reduce the number of flatfiles being sent around. Furthermore, you plan to process the data in the spatial database and create a view from the result, which will automatically update when the underlying data changes. You plan your work in 3 steps:
1. Load shapefiles and csv into PostGIS.
2. Write a spatial query to aggregate the data in PostGIS.
3. Create a view from the result of the spatial query.
4. Make sure that coworkers can access the data (e.g. via QGIS).

# 1) Load data into PostGIS (with python)

**Goal:** Create two postgis tables with the content of the NaturalEarth airport and country shapefiles.

Apply same steps to both airports and countries NaturalEarth datasets:
1. Read shapefile into geodataframes
2. Establish a connection to the target postgis database.
3. Use amazing geopandas' utility method to create new database tables from geodataframes.

#### 1. Read shapefile into geodataframes

In [16]:
import geopandas as gpd

# Airports data
airports = gpd.read_file("./data/ne_10m_airports/ne_10m_airports_csv.csv")
airports = airports.set_crs('epsg:4326').drop(columns='WKT')

In [18]:
# Country data
countries = gpd.read_file("./data/ne_50m_admin_0_countries/ne_50m_admin_0_countries.shp")

Optional: Have a quick look at the airport dataset (next 3 cells). Feel free to also explore the country data.

In [ ]:
# Show first 3 rows
airports.head(3)

In [ ]:
# Static visualization
airports.plot()

In [ ]:
# Interactive visualization
airports.explore()

#### 2. Establish a connection to the target postgis database.

In [ ]:
from sqlalchemy import create_engine

# Postgis connection credentials:
# Double check against connection info in sandbox' spatial database card!
user = 'gis'
password = 'gis'
host = 'postgis-container'
port = '5432'
dbname = 'gis'

# Establish connection
conn_string = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'
engine = create_engine(conn_string)

#### 3. Use amazing geopandas' utility method to create new database tables from geodataframes.

In [ ]:
# Create database tables from geodataframes
airports.to_postgis(
    con=engine,
    name="ne_airports",
    if_exists="replace",
    schema="public"
)
countries.to_postgis(
    con=engine,
    name="ne_countries",
    if_exists="replace",
    schema="public"
)

You should now be able to connect to the sandbox postgis database and see the new tables which hold the data - **congratulation!** The animation below shows you how to connect to postgis via pgadmin and check the new tables.

![connect to sandbox postgis and check tables](./story_images/postgis_tables.gif)

## Alternative: Load data into PostGIS (with GDAL ogr2ogr)

This section is achieving the same goal as the previous one but with a different tool. [Ogr2ogr](https://gdal.org/programs/ogr2ogr.html) is an extremely powerful GDAL utility program to convert simple features between virtually all formats via the command line. As such it is also suitable to load shapefiles to postgis. 

If you open a terminal and navigate to the location of this notebook, you can load the airports shapefile directly into a postgis table with name ne_airports_from_ogr2ogr with a single command.

```shell
ogr2ogr -f "PostgreSQL" -nln "ne_airports_from_ogr2ogr" -nlt PROMOTE_TO_MULTI --config OGR_TRUNCATE YES PG:"host=postgis-container port=5432 dbname=gis user=gis password=gis" "./data/ne_10m_airports/ne_10m_airports.shp"
```

Arguments used:
- f: Use [Postgresql/Postgis driver](https://gdal.org/drivers/vector/pg.html)
- PG: Connection details for target postgis database. Look up postgis connection credentials from sandbox' spatial database card.
- [nln](https://gdal.org/programs/ogr2ogr.html#cmdoption-ogr2ogr-nln): Target table name.
- [nlt](https://gdal.org/programs/ogr2ogr.html#cmdoption-ogr2ogr-nlt): Geometry type options for new layer. PROMOTE_TO_MULTI converts all features to multipolygons for consistency.
- [config](https://gdal.org/drivers/vector/pg.html#configuration-options): Configuration options. Set OGR_TRUNCATE to YES to erase table content before updating. Nice detail to know: Does not destroy views. 

```shell
ogr2ogr -f PostgreSQL PG:"host=postgis-container dbname=gis user=gis password=gis"  docs.csv -oo AUTODETECT_TYPE=YES -oo X_POSSIBLE_NAMES=longitude -oo Y_POSSIBLE_NAMES=latitude -oo KEEP_GEOM_COLUMNS=NO
```

??? -s_srs EPSG:4326

Arguments used:
- [open options (oo)](https://gdal.org/drivers/vector/csv.html#open-options): AUTODETECT_TYPE=YES will attempt to autodetect the type of columsn. X_POSSIBLE_NAMES and Y_POSSIBLE_NAMES can be used to specify what columns hold coordinates. KEEP_GEOM_COLUMNS=NO indicates that the coordinate columns should not be part of the attribute table.


By the way, you can even [execute shell commands](https://coderefinery.org/jupyter/extra-features/#shell-commands) directly in a jupyter notebook cell by prefixing it with an exclamation mark (!).


There is no right or wrong when it comes to which approach to use. You might prefer the python approach if your technology stack involves python scripts or you want to do preprocessing using python geo-ecosystem. In other cases you might want to prefer a command line one-liner as a quick and easy way to integrate this operation in your non-pythonic ETL stack.

# Part 2 - Data aggregation in postgis

WORK IN PROGRESS

Simple query with airport count:
```sql
SELECT c."NAME" as name, COUNT(a.geometry) AS count_airports 
FROM ne_countries AS c, ne_airports AS a
WHERE st_contains(c.geometry,a.geometry)
GROUP BY c."NAME";
```

Query with airport count and average GDP per airport:
```sql
SELECT 
    c."NE_ID" AS objectid,
    c."NAME" AS country_name, 
    COUNT(a.ne_id) AS count_airports,
    c."GDP_MD" AS gdp_musd,
    ROUND(c."GDP_MD" / COUNT(a.ne_id)) AS gdp_per_airport,
    c.geometry AS geometry
FROM ne_countries AS c, ne_airports AS a
WHERE st_contains(c.geometry,a.geometry)
GROUP BY c."NE_ID", c."NAME", c."GDP_MD", c.geometry;
```

Create a view:
```sql
CREATE or REPLACE VIEW airports_per_country AS
    SELECT 
        c."NE_ID" AS objectid,
        c."NAME" AS country_name, 
        COUNT(a.ne_id) AS count_airports,
        c."GDP_MD" AS gdp_musd,
        ROUND(c."GDP_MD" / COUNT(a.ne_id)) AS gdp_per_airport,
        c.geometry AS geometry
    FROM ne_countries AS c, ne_airports AS a
    WHERE st_contains(c.geometry,a.geometry)
    GROUP BY c."NE_ID", c."NAME", c."GDP_MD", c.geometry;
```